In [1]:
# --------------------------------------------------------------------------
# | БЛОК 1: Импорт необходимых библиотек                                   |
# --------------------------------------------------------------------------

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm
import gc 

In [2]:
# --------------------------------------------------------------------------
# | БЛОК 2: Конфигурация проекта                                           |
# --------------------------------------------------------------------------

VER = 14
CONFIG = {
    'train_path': f'C:/Users/Николай/PycharmProjects/FlightRank_2025/mydata/1/1_train.parquet',
    'test_path': f'C:/Users/Николай/PycharmProjects/FlightRank_2025/mydata/1/1_test.parquet',
    'sample_submission_path': f'C:/Users/Николай/PycharmProjects/FlightRank_2025/data/sample_submission.parquet',

    'DEVICE': 'cuda' if torch.cuda.is_available() else 'cpu', 
    'SEED': 42,
    'BATCH_SIZE': 8192*4, 
    'LR': 0.001, 
    'EPOCHS': 3, 
    'WEIGHT_DECAY': 1e-5, 

    'embedding_dims': {},
    'dropout_rate': 0.1, 
    'mlp_dims': [1024, 512, 256], 
    'num_cross_layers': 4,
    'output_dim': 1 
}

np.random.seed(CONFIG['SEED'])
torch.manual_seed(CONFIG['SEED'])
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(CONFIG['SEED'])

In [3]:
# --------------------------------------------------------------------------
# | БЛОК 3: Загрузка данных и Feature Engineering (ИСПРАВЛЕННЫЙ)           |
# --------------------------------------------------------------------------

train_df = pd.read_parquet(CONFIG['train_path'], engine='pyarrow')
test_df = pd.read_parquet(CONFIG['test_path'], engine='pyarrow')

# --- 1. Определение списков колонок ---
# ... (остается без изменений) ...
data_cols= ['legs0_arrivalAt', 'legs0_departureAt', 'legs1_arrivalAt', 'legs1_departureAt', 'requestDate']
num_cols = ['legs0_duration', 'legs0_segments0_duration', 'legs0_segments1_duration', 'legs0_segments2_duration', 'legs0_segments3_duration', 'legs1_duration', 'legs1_segments0_duration', 'legs1_segments1_duration', 'legs1_segments2_duration', 'legs1_segments3_duration', 'miniRules0_monetaryAmount', 'miniRules1_monetaryAmount', 'taxes', 'totalPrice']
bool_cols = ['isAccess3D', 'isVip', 'sex']
cat_cols = [col for col in train_df.columns if col not in data_cols and col not in num_cols and col not in bool_cols and col not in  ['ranker_id', 'selected', 'frequentFlyer']]
frequentFlyer_col = 'frequentFlyer'

# --- 2. Определение размеров эмбеддингов ---
# ... (остается без изменений) ...
for col in cat_cols:
    num_unique_values = train_df[col].nunique() + 1
    embedding_dim = int(np.sqrt(num_unique_values))
    CONFIG['embedding_dims'][col] = (num_unique_values, embedding_dim)
all_ff_codes = train_df[frequentFlyer_col].str.split('/').explode().dropna().unique()
ff_code_to_idx = {code: i for i, code in enumerate(all_ff_codes)}
ff_unknown_idx = len(ff_code_to_idx)
ff_embedding_dim = int(np.sqrt(len(all_ff_codes) + 1))
CONFIG['embedding_dims'][frequentFlyer_col] = (len(ff_code_to_idx) + 1, ff_embedding_dim)

# --- 3. ПРЕДВАРИТЕЛЬНЫЙ РАСЧЕТ ВСЕХ ГРУППОВЫХ ФИЧЕЙ ---
print("--- Предварительный расчет всех групповых фичей ---")
# Используем combined_df для всех расчетов, чтобы сохранить исходный индекс
combined_df = pd.concat([
    train_df[['ranker_id', 'totalPrice', 'legs0_duration']], 
    test_df[['ranker_id', 'totalPrice', 'legs0_duration']]
], ignore_index=True)

# Рассчитываем ранги - они привязаны к каждой строке
combined_df['price_rank'] = combined_df.groupby('ranker_id')['totalPrice'].rank(method='first').astype(int)
combined_df['duration_rank'] = combined_df.groupby('ranker_id')['legs0_duration'].rank(method='first').astype(int)

# Рассчитываем статистики и "растягиваем" их на все строки группы с помощью transform
# Это самый эффективный способ для последующего merge
group = combined_df.groupby('ranker_id')
combined_df['price_min'] = group['totalPrice'].transform('min')
combined_df['price_max'] = group['totalPrice'].transform('max')
combined_df['price_mean'] = group['totalPrice'].transform('mean')
combined_df['price_std'] = group['totalPrice'].transform('std').fillna(0)
combined_df['duration_min'] = group['legs0_duration'].transform('min')
combined_df['duration_max'] = group['legs0_duration'].transform('max')

# Создаем финальную справочную таблицу, сбрасывая исходные колонки
# Теперь group_features_df имеет тот же индекс, что и train_df/test_df
group_features_df = combined_df.drop(columns=['totalPrice', 'legs0_duration'])

# --- 4. ПРЕДРАСЧЕТ ПРЕДПОЧТЕНИЙ КОМПАНИИ ---
print("--- Предрасчет предпочтений компании по авиалиниям ---")
company_prefs = train_df[train_df['selected'] == 1].groupby('companyID')['legs0_segments0_marketingCarrier_code'].agg(lambda x: x.value_counts().index[0]).to_dict()

# --- 5. Определение списков новых фичей ---
# Ранги теперь будут числовыми фичами
new_numerical_cols = [
    'price_diff_from_min', 'price_norm_by_group', 'tax_ratio',
    'duration_diff_from_min', 'total_layover_time', 'price_rank', 'duration_rank'
]
new_bool_cols = [
    'is_cheapest', 'is_fastest', 'is_frequent_flyer_airline',
    'is_company_preferred_airline', 'is_airport_change'
]

print("Все предварительные расчеты завершены.")
del combined_df

--- Предварительный расчет всех групповых фичей ---
--- Предрасчет предпочтений компании по авиалиниям ---
Все предварительные расчеты завершены.


In [4]:
# --------------------------------------------------------------------------
# | БЛОК 4: Определение модели (с учетом всех новых фичей)                 |
# --------------------------------------------------------------------------

class FlightRankModel(nn.Module):
    def __init__(self, config, num_cols, cat_cols, bool_cols, data_cols, frequentFlyer_col, new_numerical_cols, new_bool_cols):
        super().__init__()
        # ... сохранение всех параметров ...
        self.config, self.num_cols, self.cat_cols, self.bool_cols, self.data_cols, self.frequentFlyer_col, self.new_numerical_cols, self.new_bool_cols = \
            config, num_cols, cat_cols, bool_cols, data_cols, frequentFlyer_col, new_numerical_cols, new_bool_cols

        # --- 1. ВХОДНАЯ ЧАСТЬ ---
        self.embedding_layers = nn.ModuleDict({
            col: nn.Embedding(num_embeddings=dims[0], embedding_dim=dims[1])
            for col, dims in config['embedding_dims'].items() if col != frequentFlyer_col
        })
        ff_dims = config['embedding_dims'][frequentFlyer_col]
        self.ff_embedding_layer = nn.Embedding(num_embeddings=ff_dims[0], embedding_dim=ff_dims[1])

        # Расчет размеров
        cat_embedding_dim = sum(dims[1] for col, dims in config['embedding_dims'].items() if col != frequentFlyer_col)
        ff_embedding_dim = ff_dims[1]
        numerical_dim = len(self.num_cols) + len(self.new_numerical_cols)
        boolean_dim = len(self.bool_cols) + len(self.new_bool_cols)
        cyclical_dim = len(self.data_cols) * 8
        time_to_departure_dim = 1
        
        self.input_dim = cat_embedding_dim + ff_embedding_dim + numerical_dim + boolean_dim + cyclical_dim + time_to_departure_dim
        
        self.all_numerical_batch_norm = nn.BatchNorm1d(numerical_dim + boolean_dim + cyclical_dim + time_to_departure_dim)
        
        # --- 2. CROSS & DEEP NETWORK ---
        self.cross_net = nn.ModuleList([nn.Linear(self.input_dim, self.input_dim) for _ in range(config['num_cross_layers'])])
        layer_dims = [self.input_dim] + config['mlp_dims']
        deep_layers = [nn.Sequential(nn.Linear(layer_dims[i], layer_dims[i+1]), nn.ReLU()) for i in range(len(layer_dims) - 1)]
        self.deep_net = nn.Sequential(*deep_layers)
        self.final_layer = nn.Linear(config['mlp_dims'][-1], config['output_dim'])

    def forward(self, x_dict):
        # --- 1. Формирование входного вектора x_0 ---
        embedded_features = [self.embedding_layers[col](x_dict[col]) for col in self.cat_cols]
        ff_indices, ff_mask = x_dict[self.frequentFlyer_col], x_dict[f'{self.frequentFlyer_col}_mask']
        ff_embeddings = self.ff_embedding_layer(ff_indices)
        avg_ff_embedding = torch.sum(ff_embeddings * ff_mask, dim=1) / torch.clamp(torch.sum(ff_mask, dim=1), min=1e-9)
        embedded_features.append(avg_ff_embedding)
        concatenated_embeddings = torch.cat(embedded_features, dim=1)

        # Собираем все не-эмбеддинговые признаки
        numerical_inputs = [x_dict['numerical'], x_dict['new_numerical'], x_dict['boolean'], x_dict['new_boolean']]
        for col in self.data_cols:
            date_tensor = x_dict[f'{col}_components']
            numerical_inputs.extend([torch.sin(2*np.pi*date_tensor[:, i]/d).unsqueeze(1) for i, d in [(0,59),(1,23),(2,6),(3,365)] for _ in range(2)]) # Компактная запись
            numerical_inputs[-7], numerical_inputs[-5], numerical_inputs[-3], numerical_inputs[-1] = \
                torch.cos(2*np.pi*date_tensor[:,0]/59).unsqueeze(1), torch.cos(2*np.pi*date_tensor[:,1]/23).unsqueeze(1), \
                torch.cos(2*np.pi*date_tensor[:,2]/6).unsqueeze(1), torch.cos(2*np.pi*date_tensor[:,3]/365).unsqueeze(1)
        
        time_to_departure = (x_dict['legs0_departureAt_unix'] - x_dict['requestDate_unix']) / 60.0
        numerical_inputs.append(time_to_departure.unsqueeze(1))
        
        processed_numerical_all = self.all_numerical_batch_norm(torch.cat(numerical_inputs, dim=1))
        x_0 = torch.cat([concatenated_embeddings, processed_numerical_all], dim=1)
        
        # --- 2. Cross -> Deep -> Output ---
        x_cross = x_0
        for layer in self.cross_net:
            x_cross = x_0 * torch.sigmoid(layer(x_cross)) + x_cross
        deep_output = self.deep_net(x_cross)
        return self.final_layer(deep_output)

In [5]:
# --------------------------------------------------------------------------
# | БЛОК 5: Предобработка и цикл обучения (ИСПРАВЛЕННЫЙ)                   |
# --------------------------------------------------------------------------

# --- 1. Подготовительный этап ---
print("--- Начало предобработки для обучения ---")
combined_df_num = pd.concat([train_df[num_cols], test_df[num_cols]], ignore_index=True)
num_mean = torch.tensor(combined_df_num.astype(np.float32).mean().values, dtype=torch.float32)
num_std = torch.tensor(combined_df_num.astype(np.float32).std().values, dtype=torch.float32)
num_std[num_std == 0] = 1.0 
del combined_df_num; gc.collect()
print("Статистика для нормализации рассчитана.")

# --- 2. Функция для подготовки батчей (с полным набором фичей) ---
# Определение функции get_batch остается БЕЗ ИЗМЕНЕНИЙ, оно корректно
def get_batch(df, indices, group_features, company_prefs, device):
    batch_df = df.iloc[indices].copy()
    # Получаем предрассчитанные фичи для нашего батча по тем же индексам
    batch_features = group_features.iloc[indices]
    x = {}
    epsilon = 1e-6

    # --- Создание новых фичей ---
    # Группа 1 и 2: Флаги и взаимодействия
    new_bool_features = pd.DataFrame({
        'is_cheapest': (batch_df['totalPrice'] == batch_features['price_min']).astype(float),
        'is_fastest': (batch_df['legs0_duration'] == batch_features['duration_min']).astype(float)
    })
    new_bool_features['is_frequent_flyer_airline'] = batch_df.apply(lambda row: str(row['legs0_segments0_marketingCarrier_code']) in str(row['frequentFlyer']), axis=1).astype(float)
    batch_df['company_pref_airline'] = batch_df['companyID'].map(company_prefs)
    new_bool_features['is_company_preferred_airline'] = (batch_df['legs0_segments0_marketingCarrier_code'] == batch_df['company_pref_airline']).astype(float)
    
    # Группа 3: Сложность маршрута
    new_bool_features['is_airport_change'] = ((batch_df['legs0_segments0_arrivalTo_airport_iata'] != batch_df['legs0_segments1_departureFrom_airport_iata']) & \
                                              (batch_df['legs0_segments1_departureFrom_airport_iata'] != -2)).astype(float)

    # Собираем числовые тензоры
    new_numerical_df = pd.DataFrame({
        'price_diff_from_min': batch_df['totalPrice'] - batch_features['price_min'],
        'price_norm_by_group': (batch_df['totalPrice'] - batch_features['price_mean']) / (batch_features['price_std'] + epsilon),
        'tax_ratio': batch_df['taxes'] / (batch_df['totalPrice'] + epsilon),
        'duration_diff_from_min': batch_df['legs0_duration'] - batch_features['duration_min'],
        'total_layover_time': batch_df['legs0_duration'] - batch_df[['legs0_segments0_duration', 'legs0_segments1_duration', 'legs0_segments2_duration', 'legs0_segments3_duration']].sum(axis=1),
        'price_rank': batch_features['price_rank'],
        'duration_rank': batch_features['duration_rank']
    })
    
    x['new_numerical'] = torch.tensor(new_numerical_df[new_numerical_cols].fillna(0).values.astype(np.float32), device=device)
    x['new_boolean'] = torch.tensor(new_bool_features[new_bool_cols].fillna(0).values.astype(np.float32), device=device)
    
    # --- Обработка остальных фичей (код без изменений) ---
    for col in cat_cols: x[col] = torch.tensor(batch_df[col].values.astype(np.int64), dtype=torch.long, device=device)
    numerical_tensor = torch.tensor(batch_df[num_cols].values.astype(np.float32), device=device)
    x['numerical'] = (numerical_tensor - num_mean.to(device)) / num_std.to(device)
    x['boolean'] = torch.tensor(batch_df[bool_cols].values.astype(np.float32), device=device)
    for col in data_cols:
        dt_series = pd.to_datetime(batch_df[col], errors='coerce')
        x[f'{col}_components'] = torch.tensor(np.vstack([dt_series.dt.minute.fillna(0), dt_series.dt.hour.fillna(0), dt_series.dt.dayofweek.fillna(0), dt_series.dt.dayofyear.fillna(0)]).T, dtype=torch.float32, device=device)
        x[f'{col}_unix'] = torch.tensor(dt_series.astype(np.int64).values // 10**9, dtype=torch.float32, device=device)
    ff_str_list = batch_df[frequentFlyer_col].fillna('').tolist()
    list_of_indices = [[ff_code_to_idx.get(code, ff_unknown_idx) for code in s.split('/') if code] for s in ff_str_list]
    max_len = max(len(s) for s in list_of_indices) if any(list_of_indices) else 1
    indices_tensor = torch.full((len(batch_df), max_len), ff_unknown_idx, dtype=torch.long, device=device)
    mask_tensor = torch.zeros(len(batch_df), max_len, 1, dtype=torch.float32, device=device)
    for i, codes in enumerate(list_of_indices):
        if codes:
            indices_tensor[i, :len(codes)] = torch.tensor(codes, dtype=torch.long)
            mask_tensor[i, :len(codes)] = 1.0
    x[frequentFlyer_col], x[f'{frequentFlyer_col}_mask'] = indices_tensor, mask_tensor
    y = torch.tensor(batch_df['selected'].values.astype(float), dtype=torch.float32, device=device).unsqueeze(1) if 'selected' in batch_df.columns else None
    return x, y

# --- 3. Инициализация и цикл обучения ---
model = FlightRankModel(CONFIG, num_cols, cat_cols, bool_cols, data_cols, frequentFlyer_col, new_numerical_cols, new_bool_cols).to(CONFIG['DEVICE'])
criterion = nn.BCEWithLogitsLoss()
optimizer = Adam(model.parameters(), lr=CONFIG['LR'])

print(f"\n--- Начало обучения на {CONFIG['EPOCHS']} эпох ---")
for epoch in range(CONFIG['EPOCHS']):
    model.train()
    running_loss = 0.0
    shuffled_indices = np.random.permutation(len(train_df))
    num_batches = (len(train_df) + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']
    
    progress_bar = tqdm(range(num_batches), desc=f"Эпоха {epoch + 1}/{CONFIG['EPOCHS']}")
    
    for i in progress_bar:
        batch_indices = shuffled_indices[i * CONFIG['BATCH_SIZE'] : (i + 1) * CONFIG['BATCH_SIZE']]
        # ИСПРАВЛЕНИЕ: используем правильное имя переменной `group_features_df`
        x_batch, y_batch = get_batch(train_df, batch_indices, group_features_df, company_prefs, CONFIG['DEVICE'])
        
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        current_avg_loss = running_loss / (i + 1)
        progress_bar.set_postfix(avg_loss=f'{current_avg_loss:.4f}')
        
    print(f"Итоговый средний лосс за эпоху {epoch + 1}: {running_loss / num_batches:.4f}")

# --- 4. Цикл предсказания ---
print("\n--- Генерация предсказаний для теста ---")
model.eval()
test_preds = []
# Совмещаем test_df с его предрассчитанными фичами для предсказания
# Это нужно сделать один раз, чтобы индексы совпадали
test_with_features = test_df.join(group_features_df.drop(columns=['ranker_id']))
num_test_batches = (len(test_with_features) + CONFIG['BATCH_SIZE'] - 1) // CONFIG['BATCH_SIZE']

with torch.no_grad():
    for i in tqdm(range(num_test_batches), desc="Предсказание"):
        test_indices = list(range(i * CONFIG['BATCH_SIZE'], min((i + 1) * CONFIG['BATCH_SIZE'], len(test_with_features))))
        # ИСПРАВЛЕНИЕ: используем правильное имя переменной `group_features_df`
        x_batch, _ = get_batch(test_with_features, test_indices, group_features_df.iloc[len(train_df):].reset_index(drop=True), company_prefs, CONFIG['DEVICE'])
        
        outputs = model(x_batch)
        preds = torch.sigmoid(outputs).cpu().numpy().flatten()
        test_preds.extend(preds)

test_df['score'] = test_preds

--- Начало предобработки для обучения ---
Статистика для нормализации рассчитана.

--- Начало обучения на 3 эпох ---


Эпоха 1/3: 100%|██████████| 554/554 [25:39<00:00,  2.78s/it, avg_loss=0.0253]


Итоговый средний лосс за эпоху 1: 0.0253


Эпоха 2/3: 100%|██████████| 554/554 [22:47<00:00,  2.47s/it, avg_loss=0.0208]


Итоговый средний лосс за эпоху 2: 0.0208


Эпоха 3/3: 100%|██████████| 554/554 [28:20<00:00,  3.07s/it, avg_loss=0.0196]


Итоговый средний лосс за эпоху 3: 0.0196

--- Генерация предсказаний для теста ---


Предсказание: 100%|██████████| 211/211 [06:28<00:00,  1.84s/it]


In [6]:
# --------------------------------------------------------------------------
# | БЛОК 6: Формирование файла для отправки (submission)                   |
# --------------------------------------------------------------------------
# --------------------------------------------------------------------------

print("\n--- Формирование файла для отправки ---")

# Обновляем CONFIG, чтобы указать путь для сохранения
CONFIG['submission_path'] = f'C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_{VER}.csv'

sample_submission_df = pd.read_parquet(CONFIG['sample_submission_path'])
test_df['Id'] = sample_submission_df['Id'].values


# Ранжирование
# Группируем по ranker_id и ранжируем строки внутри каждой группы по 'score'.
# ascending=False, так как более высокий скор означает лучший ранг (ранг 1).
# method='first' гарантирует, что не будет одинаковых рангов в группе.
test_df['selected'] = test_df.groupby('ranker_id')['score'].rank(method='first', ascending=False).astype(int)

# Создаем файл для отправки
submission_df = test_df[['Id', 'ranker_id', 'selected']]

# Убедимся, что порядок строк соответствует исходному файлу
submission_df = submission_df.set_index('Id').loc[sample_submission_df['Id']].reset_index()

submission_df.to_csv(CONFIG['submission_path'], index=False)

print(f"\nГотово! Файл для отправки сохранен в: {CONFIG['submission_path']}")
print("Пример содержимого submission файла:")
print(submission_df.head())


--- Формирование файла для отправки ---


C:\Users\Николай\AppData\Local\Temp\ipykernel_9448\4176084543.py:19: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_df['selected'] = test_df.groupby('ranker_id')['score'].rank(method='first', ascending=False).astype(int)



Готово! Файл для отправки сохранен в: C:/Users/Николай/PycharmProjects/FlightRank_2025/submissions/submission_14.csv
Пример содержимого submission файла:
         Id                         ranker_id  selected
0  18144679  c9373e5f772e43d593dd6ad2fa90f67a        46
1  18144680  c9373e5f772e43d593dd6ad2fa90f67a        60
2  18144681  c9373e5f772e43d593dd6ad2fa90f67a       205
3  18144682  c9373e5f772e43d593dd6ad2fa90f67a        80
4  18144683  c9373e5f772e43d593dd6ad2fa90f67a        50


In [7]:
test_df['score'][1]

np.float32(0.007349402)